In [1]:
import numpy as np
import os

from dingodb import SDKVectorDingoDB, SDKClient
from dingodb.common.vector_rep import ScalarType, ScalarColumn, ScalarSchema

addrs = "172.30.14.11:22001,172.30.14.11:22002,172.30.14.11:22003"
sdk_client = SDKClient(addrs)
x = SDKVectorDingoDB(sdk_client)
print(x)


index_name = "test_index_grpc"

In [2]:
# delete_index
x.delete_index(index_name)

True

In [3]:
# create_index
# help(x.create_index)
x.create_index(index_name, 6, "diskann", "euclidean", 3, index_config={"valueType": "float","searchListSize": 100,"maxDegree": 64}, operand=[5,10,15,20])


True

In [4]:
# make dataset
# 构建训练数据和查询数据
d = 6                           # dimension
nb = 4                      # database size
np.random.seed(1234)             # make reproducible
xb = np.random.random((nb, d)).astype('float32')
print(xb)
print(xb.shape)
xb[:, 0] += np.arange(nb) / 1000.
print(xb)
print(xb.shape)

ids = [1, 2, 3, 4]
datas = [{"a1": "b1"}, {"a2": "b2"}, {"a3": "b3"}, {"a4": "b4"}]
vectors = xb.tolist()

[[0.19151945 0.62210876 0.43772775 0.7853586  0.77997583 0.2725926 ]
 [0.27646425 0.8018722  0.95813936 0.87593263 0.35781726 0.5009951 ]
 [0.6834629  0.71270204 0.37025076 0.5611962  0.50308317 0.01376845]
 [0.7728266  0.8826412  0.364886   0.6153962  0.07538124 0.368824  ]]
(4, 6)
[[0.19151945 0.62210876 0.43772775 0.7853586  0.77997583 0.2725926 ]
 [0.27746424 0.8018722  0.95813936 0.87593263 0.35781726 0.5009951 ]
 [0.6854629  0.71270204 0.37025076 0.5611962  0.50308317 0.01376845]
 [0.77582663 0.8826412  0.364886   0.6153962  0.07538124 0.368824  ]]
(4, 6)


In [5]:
# vector_add
x.vector_import_add(index_name, datas, vectors, ids)
x.vector_import_add(index_name, datas, vectors, ids)
x.vector_import_add(index_name, datas, vectors, ids)
x.vector_import_add(index_name, datas, vectors, ids)

[{'id': 13,
  'vector': {'dimension': 6,
   'valueType': 'FLOAT',
   'floatValues': [0.19151945412158966,
    0.6221087574958801,
    0.43772774934768677,
    0.7853586077690125,
    0.7799758315086365,
    0.27259260416030884],
   'binaryValues': []},
  'scalarData': {'a1': {'fieldType': 'STRING', 'fields': ['b1']}}},
 {'id': 14,
  'vector': {'dimension': 6,
   'valueType': 'FLOAT',
   'floatValues': [0.27746424078941345,
    0.801872193813324,
    0.9581393599510193,
    0.8759326338768005,
    0.35781726241111755,
    0.5009950995445251],
   'binaryValues': []},
  'scalarData': {'a2': {'fieldType': 'STRING', 'fields': ['b2']}}},
 {'id': 15,
  'vector': {'dimension': 6,
   'valueType': 'FLOAT',
   'floatValues': [0.6854628920555115,
    0.7127020359039307,
    0.37025076150894165,
    0.5611962080001831,
    0.5030831694602966,
    0.013768449425697327],
   'binaryValues': []},
  'scalarData': {'a3': {'fieldType': 'STRING', 'fields': ['b3']}}},
 {'id': 16,
  'vector': {'dimension': 6

In [6]:
x.vector_build_by_index(index_name)

[]

In [7]:
[r.to_dict() for r in x.vector_status_by_index(index_name)]

[{'region_id ': 80001, 'region_state': 'Builded'},
 {'region_id ': 80002, 'region_state': 'Builded'},
 {'region_id ': 80003, 'region_state': 'Builded'},
 {'region_id ': 80004, 'region_state': 'Builded'},
 {'region_id ': 80005, 'region_state': 'NoData'}]

In [8]:
x.vector_count_memory(index_name)

16

In [9]:
[r.to_dict() for r in x.vector_load_by_index(index_name)]

[]

In [10]:
x.vector_search(index_name, vectors[0])

[{'vectorWithDistances': [{'id': 1,
    'vector': {'dimension': 6,
     'valueType': 'FLOAT',
     'floatValues': [0.19151945412158966,
      0.6221087574958801,
      0.43772774934768677,
      0.7853586077690125,
      0.7799758315086365,
      0.27259260416030884],
     'binaryValues': []},
    'scalarData': {'a1': {'fieldType': 'STRING', 'fields': ['b1']}},
    'distance': 0.0,
    'metricType': 'METRIC_TYPE_L2'},
   {'id': 13,
    'vector': {'dimension': 6,
     'valueType': 'FLOAT',
     'floatValues': [0.19151945412158966,
      0.6221087574958801,
      0.43772774934768677,
      0.7853586077690125,
      0.7799758315086365,
      0.27259260416030884],
     'binaryValues': []},
    'scalarData': {'a1': {'fieldType': 'STRING', 'fields': ['b1']}},
    'distance': 0.0,
    'metricType': 'METRIC_TYPE_L2'},
   {'id': 5,
    'vector': {'dimension': 6,
     'valueType': 'FLOAT',
     'floatValues': [0.19151945412158966,
      0.6221087574958801,
      0.43772774934768677,
      0.7853

In [11]:
[r.to_dict() for r in x.vector_reset_by_index(index_name)]

[]

In [16]:
id = [1,80001]
[r.to_dict() for r in x.vector_status_by_region(index_name,id)]

RuntimeError: vector status by region  form index:test_index_grpc fail: NotFound (errno:60001) : region_id not exists

In [15]:
[r.to_dict() for r in x.vector_build_by_region(index_name,id)]

[{'region_id': 1, 'err_msg': 'NotFound (errno:60001) : region_id not exists'}]

In [17]:
[r.to_dict() for r in x.vector_load_by_region(index_name,id)]

[{'region_id': 1, 'err_msg': 'NotFound (errno:60001) : region_id not exists'}]

In [18]:
[r.to_dict() for r in x.vector_reset_by_region(index_name,id)]

[{'region_id': 1, 'err_msg': 'NotFound (errno:60001) : region_id not exists'}]

In [19]:
x.vector_import_delete(index_name,ids)

In [20]:
x.delete_index(index_name)

True